# Review
## from 02. Autograd-자동미분 ~ 
20.03.31.tue.pm 6:12

* List
    * 02. Autograd
    * 03. 신경망(Neural Networks)

<code>autograd</code>패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공한다.<br>
define-by-run 프레임워크로, 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻이며, 역전파는 학습 과정의 매 단계마다 달라진다.

In [1]:
import torch

### Tensor

tensor를 생성하고, <code>requires_grad=True</code>를 설정하여 연산을 기록한다.

In [2]:
x = torch.ones(2, 2, requires_grad=True)  # requires_grad=True로 설정하여 연산을 기록한다.
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
# tensor에 연산을 수행한다.

y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


y는 연산 결과로 생성된 것이므로 <code>.grad_fn</code>을 갖는다.

In [4]:
print(y.grad_fn)

<code>.grad_fn</code>속성은 <code>Tensor</code>를 생성한 <code>Function</code>을 참조하고 있다.

<code>y</code>에 다른 연산을 수행해보자

In [14]:
z = y * y * 3
out = z.mean()

print(z, out)  # z 또한 .grad_fn 속성을 가지고 있을까?

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


<code>.requires_grad_(...)</code>는 기존 Tensor의 <code>requires_grad</code>값을 바꿔치기 (in-place)하여 변경한다.<br>
(기존 requires_grad였던 x를 바꿔치기 한다는 의미인가?)<br>


a라는 새로운 tensor를 만들어보자

In [7]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)

False


<code>a.requires_grad</code>를 출력해보면 False로 출력되는 것을 볼 수 있다.<br>
<code>requires_grad</code>를 True로 변경하기 위해 <code>.requires_grad_(...)</code>를 사용하여 기존 값을 in-place한다.

In [9]:
a.requires_grad_(True)  # .requires_grad_(True)로 인해 기존의 requires_grad=False가 True로 바뀌었다.
print(a)

tensor([[  0.6380,  -0.5452],
        [-12.0294,   0.9535]], requires_grad=True)


<code>requires_grad</code>가 True로 변경된 것을 볼 수 있다.

In [10]:
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)  # b의 grad_fn은 Tensor를 생성한 Function을 참조한다. 

True


### Gradient 

이제 역전파(backprop)도 해보자.
앞서 정의한 변수 <code>out</code>은 하나의 스칼라 값만 갖고 있기 때문에 <code>out.backward()</code>는 <code>out.backward(torch.tensor(1.))</code>과 동일하다.<br>
(out은 z의 평균값으로 위에서 구현해 두었다) 

In [15]:
out.backward()

변화도 d(out)/dx를 출력한다.

In [17]:
print(x.grad)

tensor([[9., 9.],
        [9., 9.]])


벡터-야코비안 곱의 예제를 살펴보도록 하자.

* 일반적으로 , <code>torch.autograd</code>는 벡터-야코비안 곱을 계산하는 엔진이다. 

    즉, 어떤 벡터 v=(v1v2⋯vm)T 에 대해 vT⋅J 을 연산합니다. 만약 v 가 스칼라 함수 l=g(y⃗ ) 의 기울기인 경우, v=(∂l∂y1⋯∂l∂ym)T 이며, 연쇄법칙(chain rule)에 따라 벡터-야코비안 곱은 x⃗  에 대한 l 의 기울기가 됩니다:

In [31]:
x = torch.randn(3, requires_grad=True)
print('x: ', x)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2
    
print('y: ', y)

x:  tensor([0.1111, 1.6964, 0.8650], requires_grad=True)
y:  tensor([ 113.7798, 1737.1522,  885.8029], grad_fn=<MulBackward0>)


<code>torch.autograd</code>는 전체 야코비안을 직접 계산할수는 없지만, 벡터-야코비안 곱은 간단히 <code>backward</code>에 해당 벡터를 인자로 제공하여 얻을 수 있다.

In [32]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [33]:
print(y.grad)

None


In [34]:
print(v.grad)

None


In [ ]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)  # v에 대한 벡터-야코비안 곱을 계산한다. 


print(x.grad)

Q. Jacobian-Matrix란?

* cs231n에서도 매우 혼동되었던 개념!!  - 20.03.31.Tue - 
    * <code>채워넣기</code>


In [31]:
print(x.requires_grad)
print((x ** 2).requires_grad)

True
True


In [32]:
with torch.no_grad():
    print((x ** 2).requires_grad)

False


In [36]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)


## 03. 신경망 

신경망은 <code>torch.nn</code>패키지를 사용하여 생성할 수 있다.

<code>nn</code>은 모델을 정의하고 미분하는데 <code>autograd</code>를 사용한다. <code>nn.Module</code>은 계층(layer)과 <code>output</code>을 반환하는 <code>forward(input)</code>메서드를 포함하고 있다.

간단한 순전파 네트워크(Feed-forward network)를 구현한다. 입력(input)을 받아 여러 계층에 차례로 전달한 후, 최종 출력(output)을 제공한다.

* 신경망의 일반적인 학습과정
    * <code>채워넣기</code>

In [48]:
import torch
import torch.nn as  nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channer, 6 output channels, 3x3 square convolution
        # kernel
        
        # 참고 https://github.com/inmoonlight/PyTorchTutorial/blob/master/01_CNN.ipynb
        
                                         # 희한하네 
                                         # 
        self.conv1 = nn.Conv2d(1, 6, 3)  # Conv2d 통과 후 receptive field 값.
                                         # 초기 값이 그림과 같이 28*28일 때,  6@26*26
                                         # 튜토리얼이라 그런지 pool레이어가 생략되어있다(여기서 혼동이 온듯)
                                         # 6@13*13
        self.conv2 = nn.Conv2d(6, 16, 3) # 16@11*11
                                         # 16@6*6
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6 * 6 from Image dimension # ???
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [49]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


<code>forward</code>함수만 정의하고 나면, (변화도를 계산하는) <code>backward</code>함수는 <code>autograd</code>를 사용하여 자동으로 정의된다. <code>forward</code>함수에서는 어떠한 Tensor 연산을 사용해도 된다.

모델의 학습 가능한 매개변수들은 <code>net.parameters()</code>에 의해 반환된다.

In [50]:
params = list(net.parameters())
# print(params)
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


임의의 32x32 입력값을 넣어본다.<br>
**Note** <br>
이 신경망(LeNet)의 예상되는 입력의 크기는 32x32이다.<br>
이 신경망에 MNIST 데이터셋을 사용하기 위해서는 데이터셋의 이미지 크기를 32x32로 변경해야 한다.

그러면 위에서 self.f1 = nn.Linear(16 * 6 * 6, 120)의 6 * 6은 대체 뭐야??

In [52]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.1898, -0.0190,  0.0888,  0.0708, -0.1248, -0.0199,  0.0415,  0.0632,
         -0.0576,  0.0231]], grad_fn=<AddmmBackward>)


모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정하고, 무작위 값으로 역전파를 한다.

In [53]:
net.zero_grad()
out.backward(torch.randn(1, 10))

### 손실 함수(Loss Funciton)

In [55]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9750, grad_fn=<MseLossBackward>)


In [59]:
print(loss.grad_fn)  # MSE loss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # Relu

### 역전파(Backprop)

In [60]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0021, -0.0098, -0.0075,  0.0100, -0.0083,  0.0001])


In [61]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [62]:
import torch.optim as optim

# Optimizer를 생성합니다.
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정(training loop)에서는 다음과 같습니다:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update